In [1]:
import pandas as pd
import numpy as np

In [26]:
# Data paths
user_ratings_path = "../Datasets/Ratings.csv"
user_books_path = "../Datasets/Books.csv"
user_users_path = "../Datasets/Users.csv"

# Load data
user_ratings = pd.read_csv(user_ratings_path)
user_books = pd.read_csv(user_books_path)
user_users = pd.read_csv(user_users_path)


# Add ratings to the user_books
ratings_summary = user_ratings.groupby("ISBN")["Book-Rating"].agg(["count", "mean"])
ratings_summary = ratings_summary.rename(columns={"count": "Rating_Count", "mean": "Average_Rating"})
user_books = pd.merge(user_books, ratings_summary, on="ISBN", how="left")
user_books["Rating_Count"] = user_books["Rating_Count"].fillna(0) 
user_books["Average_Rating"] = user_books["Average_Rating"].fillna(user_books["Average_Rating"].median())

#Remove photos
user_books.drop(inplace=True, axis=1, columns=["Image-URL-S", "Image-URL-M", "Image-URL-L"])


C:\Users\Ivans\AppData\Local\Temp\ipykernel_31084\2952673529.py:8: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  user_books = pd.read_csv(user_books_path)


In [3]:
model_path = "../models/books_embeddings_new_dataset.npy"

#Load the model
model = np.load(model_path, allow_pickle=True)

In [23]:
user_books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Rating_Count,Average_Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,1.0,0.000000
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,14.0,4.928571
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,3.0,5.000000
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,11.0,4.272727
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,1.0,0.000000


In [30]:
user_ratings["Book-Title"] = user_ratings["ISBN"].map(user_books.set_index("ISBN")["Book-Title"])
grouped = user_ratings.groupby('User-ID').apply(lambda x: list(zip(x['Book-Title'], x['Book-Rating']))).reset_index(name='ISBN_Rating')
grouped.head()

C:\Users\Ivans\AppData\Local\Temp\ipykernel_31084\4006359190.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = user_ratings.groupby('User-ID').apply(lambda x: list(zip(x['Book-Title'], x['Book-Rating']))).reset_index(name='ISBN_Rating')


,User-ID,ISBN_Rating
0,2,"[(Classical Mythology, 0)]"
1,7,"[(nan, 0)]"
2,8,"[(Clara Callan, 5), (Decision in Normandy, 0),..."
3,9,"[(The Testament, 0), (Beloved (Plume Contempor..."
4,10,[(New Vegetarian: Bold and Beautiful Recipes f...


In [49]:
model[0][0]

'Classical Mythology'

In [53]:
model_dict = {}
for book_index in range(len(model)):
	book = model[book_index]
	model_dict[book[0]] = book_index

In [52]:
model_dict.keys()

dict_keys(['Classical Mythology', 'Decision in Normandy', 'Flu', 'Under the Black Flag', 'The Middle Stories', 'More Cunning Than Man', 'The Testament', 'Beloved', 'Our Dumb Century', 'New Vegetarian', "Tell Me This Isn't Happening!", 'Flood', 'Wild Animus', 'Airframe', 'Timeline', 'Out of the Silent Planet', 'Prague', 'Chocolate Jesus', 'Sturmzeit', 'Lying Awake', 'To Kill a Mockingbird', 'Seabiscuit', 'Pigs in Heaven', "Miss Zukas and the Raven's Dance", 'Pride and Prejudice', 'Therapeutic Touch', 'Downtown', 'Icebound', "I'll Be Seeing You", 'From the Corner of His Eye', 'Isle of Dogs', 'Purity in Death', 'This Year it Will be Different and Other Stories', 'Left Behind', 'The Street Lawyer', "All the King's Men", 'A Soldier of the Great War', 'Getting Well Again', 'An Atmosphere of Eternity', 'Lonesome Dove', 'Shabanu', 'Haveli', 'Lieux dits', 'The Dragons of Eden', 'The Joy Luck Club', 'Heart of Darkness and Other Stories', 'The Tao of Pooh', 'The Prince', "Life's Little Instructio

In [ ]:
def calculate_embedding(ratings):
	resulting = 0
	for book in ratings:
		if book[0] in model_dict:
			emb = model[model_dict[book[0]]][1:]
			resulting += emb * (5-book[1]) * 0.1

	return resulting
		

grouped["result"] = grouped["ISBN_Rating"].apply(calculate_embedding)

In [58]:
grouped

,User-ID,ISBN_Rating,result
0,2,"[(Classical Mythology, 0)]","[-0.08982004970312119, 0.046994734555482864, 0..."
1,7,"[(nan, 0)]",0
2,8,"[(Clara Callan, 5), (Decision in Normandy, 0),...","[-0.1983802616596222, -0.006100172642618418, -..."
3,9,"[(The Testament, 0), (Beloved (Plume Contempor...","[-0.07386114448308945, 0.040893182158470154, 0..."
4,10,[(New Vegetarian: Bold and Beautiful Recipes f...,0
...,...,...,...
105278,278846,"[(Brave New World, 8), (nan, 0)]","[0.041135452687740326, -0.054563520848751074, ..."
105279,278849,[(Behind the Attic Wall (Avon Camelot Books (P...,0
105280,278851,[(Frommer's 2000 California (Frommer's Califor...,"[-0.022739240899682045, 0.0661618709564209, 0...."
105281,278852,[(Dave Barry's Only Travel Guide You'll Ever N...,"[-0.005927275121212006, -0.08430862426757812, ..."
